In [45]:
import numpy as np
import pandas as pd
import psycopg2
import json
import uuid

Prerequisites:
- Having classes.txt file in the same root folder
- Having the database up and running

In [2]:
#get classification data from database
cs = "dbname=%s user=%s password=%s host=%s port=%s" % ("FilmColors_v2_Production","ERCAdmin","admin","localhost","5432")
conn = psycopg2.connect(cs)
cur = conn.cursor()

cur.execute("SELECT id, video_id, classification_object, analysis_class_name, uuid, dtype, shape, data FROM public.db_concert_analyses where analysis_class_name='ClassificationAnalysis';")
recs = cur.fetchall()

cur.close()
conn.close()

In [47]:
with open('classes.txt', 'r') as f:
    classes = [line.rstrip('\n') for line in f]

classes = np.array(classes)

groups = [134, 147, 156, 179, 183, 189, 194]
group_index = [x - 134 for x in groups]

for row in recs:
    print(row[1])
    averaged_predictions = np.frombuffer(row[7], dtype=np.float64)
    musical_instruments_vector = averaged_predictions[134:203]
    musical_instruments_vector = np.delete(musical_instruments_vector, 45)

    musical_instruments_classes = classes[134:203]
    musical_instruments_classes = np.delete(musical_instruments_classes, 45)

    for i, l in enumerate(musical_instruments_vector[group_index].argsort()[-1:][::-1], 1):
        family = musical_instruments_classes[group_index][l].strip().replace("\"", "")

    inst = []
    instrument_count = 1
    for i, l in enumerate(musical_instruments_vector.argsort()[-13:][::-1], 1):
        if l not in group_index and instrument_count < 3:
            instrument_count +=1
            inst.append(musical_instruments_classes[l].strip().replace("\"", "").replace(",", ""))
    
    genre_vector = averaged_predictions[211:271]
    genre_classes = classes[211:271]

    mood_vector = averaged_predictions[271:277]
    mood_classes = classes[271:277]

    genre = []
    for i, l in enumerate(genre_vector.argsort()[-2:][::-1], 1):
        genre.append(genre_classes[l].strip().replace("\"", "").replace(",", ""))
    

    for i, l in enumerate(mood_vector.argsort()[-1:][::-1], 1):
        mood = mood_classes[l].strip().replace("\"", "").replace(",", "")
    
    
    data = {"family":family,  "instrument":inst, "genre":genre, "mood": mood}

    dumped_json_string = json.dumps(data)
    binary_data = ' '.join(format(ord(letter), 'b') for letter in dumped_json_string)
    print(dumped_json_string)
    

    cs = "dbname=%s user=%s password=%s host=%s port=%s" % ("FilmColors_v2_Production","ERCAdmin","admin","localhost","5432")
    conn = psycopg2.connect(cs)
    cur = conn.cursor()

    cur.execute("INSERT INTO public.db_concert_analyses(video_id, classification_object, analysis_class_name, uuid, dtype, shape, data) VALUES (%s, %s, %s, %s, %s, %s, %s)"
            ,(row[1], 'Global', "MetadataAnalysis",str(uuid.uuid1()), "dict", "",binary_data))
    conn.commit()
    cur.close()
    conn.close()

586
{"family": "Percussion,", "instrument": ["Mallet percussion", "Guitar"], "genre": ["Jazz", "Blues"], "mood": "Tender music"}
585
{"family": "Percussion,", "instrument": ["Drum", "Flute"], "genre": ["Ska", "Country"], "mood": "Happy music"}
584
{"family": "Percussion,", "instrument": ["Drum", "Drum kit"], "genre": ["Ska", "Reggae"], "mood": "Happy music"}
583
{"family": "Plucked string instrument,", "instrument": ["Guitar", "Acoustic guitar"], "genre": ["Country", "Bluegrass"], "mood": "Tender music"}
582
{"family": "Brass instrument,", "instrument": ["Saxophone", "Trumpet"], "genre": ["Jazz", "Swing music"], "mood": "Tender music"}
581
{"family": "Plucked string instrument,", "instrument": ["Guitar", "Acoustic guitar"], "genre": ["Soul music", "Christian music"], "mood": "Tender music"}
580
{"family": "Bell,", "instrument": ["Church bell", "Guitar"], "genre": ["Rock music", "Heavy metal"], "mood": "Exciting music"}
579
{"family": "Plucked string instrument,", "instrument": ["Guitar